In [1]:
import os

In [2]:
%pwd

'd:\\LoanTap\\LoanTap\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    model_score: Path
    scaler_path: Path
    target_column: str
    


In [5]:
from src.LoanTapPred.constants import *
from src.LoanTapPred.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            model_score=config.model_score,
            scaler_path=config.scaler_path,
            target_column=schema.name

        )
        return model_trainer_config

In [6]:
import os
from src.LoanTapPred import logger
import pandas as pd
from sklearn.linear_model import LogisticRegression
from src.LoanTapPred.utils.common import save_bin
from src.LoanTapPred.utils.data_transformation_utils import scale_data
import joblib
from imblearn.over_sampling import SMOTE

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]
        
        X_test = test_data.drop(columns=[self.config.target_column])
        y_test = test_data[self.config.target_column]
        
        X_train_scaled=scale_data(X_train, os.path.join(self.config.root_dir, self.config.scaler_path))
        
        sm=SMOTE(random_state=42)
        X_train_res,y_train_res=sm.fit_resample(X_train_scaled,y_train)
        lr=LogisticRegression()
        lr.fit(X_train_res,y_train_res)
        print(lr.score(X_train_res,y_train_res))
        with open(os.path.join(self.config.model_score),'w') as f:
            f.write(f'Model score is: {lr.score(X_train_res,y_train_res)}')

        
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


        





    

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    moderl_trainer = ModelTrainer(config=model_trainer_config)
    moderl_trainer.train()
except Exception as e:
    raise e

[2025-01-22 23:52:15,692: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-22 23:52:15,692: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-22 23:52:15,692: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-22 23:52:15,707: INFO: common: created directory at: artifacts]
[2025-01-22 23:52:15,709: INFO: common: created directory at: artifacts/model_trainer]
0.806949822111571
